In [70]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
X = [[0, 0], [1, 1], [1, 0], [0, 1]]
y = [0, 0, 1, 1]
ts = TensorSketch(degree=2, n_components=100)
X_features = ts.fit_transform(X)
clf = LinearSVC()
clf.fit(X_features, y)



LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [69]:

clf.score(X_features, y)

1.0

In [53]:
import warnings

import numpy as np
import scipy.sparse as sp
from scipy.linalg import svd
from scipy import fftpack

from sklearn.base import  BaseEstimator, TransformerMixin
from sklearn.utils import check_array, check_random_state, as_float_array
from sklearn.utils.validation import  check_is_fitted
        
class TensorSketch( BaseEstimator, TransformerMixin):
    """Tensor Sketch [1] approximates the feature map of the homogeneous polynomial kernel by efficiently computing
    a Count Sketch [2] of the outer product of a vector with itself.

    Parameters
    ----------
    degree : int
        Degree of the homogeneous polynomial kernel whose feature map will be approximated.
        The kernel is K(x,y) = <x,y>^d.

    n_components : int
        Dimensionality of the output feature space.

    random_state : int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`.

    Examples
    --------
    >>> from sklearn.kernel_approximation import TensorSketch
    >>> from sklearn.linear_model import SGDClassifier
    >>> X = [[0, 0], [1, 1], [1, 0], [0, 1]]
    >>> y = [0, 0, 1, 1]
    >>> ts = TensorSketch(degree=2, random_state=1)
    >>> X_features = ts.fit_transform(X)
    >>> clf = SGDClassifier(max_iter=5, tol=1e-3)
    >>> clf.fit(X_features, y)
    ... # doctest: +NORMALIZE_WHITESPACE
    SGDClassifier(alpha=0.0001, average=False, class_weight=None,
           early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
           l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
           n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
           power_t=0.5, random_state=None, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)
    >>> clf.score(X_features, y)
    1.0

    Notes
    -----

    [1]Pham, N., & Pagh, R. (2013, August). Fast and scalable polynomial kernels via explicit feature maps. 
    In Proceedings of the 19th ACM SIGKDD international conference on Knowledge discovery and data mining (pp. 239-247). ACM.
    (https://pdfs.semanticscholar.org/76ca/15e975b0dee581d5d04dca26dfae636372de.pdf)
    
    [2] Charikar, M., Chen, K., & Farach-Colton, M. (2002, July). Finding frequent items in data streams. 
    In International Colloquium on Automata, Languages, and Programming (pp. 693-703). Springer, Berlin, Heidelberg.
    (http://www.vldb.org/pvldb/1/1454225.pdf)
    
    """

    def __init__(self, degree=2, n_components=100, random_state=None):
        
        self.degree = degree
        self.n_components = n_components
        self.random_state = random_state
    
    def fit(self, X, Y=None):
        """Fit the model with X.

        Initializes the internal variables. The method is data-independent, so we only care about
        the dimension of samples in X, not their distribution.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            Training data, where n_samples in the number of samples
            and n_features is the number of features.

        Returns
        -------
        self : object
            Returns the transformer.
        """
        
        X = check_array(X, accept_sparse='csr')
        random_state = check_random_state(self.random_state)
        n_features = X.shape[1]
        
        self.indexHash_ = random_state.randint(0, high=self.n_components,size=(self.degree, n_features))
        self.bitHash_ = random_state.choice(a=[-1.,1.], size=(self.degree, n_features)).astype(np.float32)
        
        return self
    

    

    def transform(self, X, Y=None):
        """Generate the feature map approximation for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            New data, where n_samples in the number of samples
            and n_features is the number of features.

        Returns
        -------
        X_new : array-like, shape (n_samples, n_components)
        """

        check_is_fitted(self, 'indexHash_')
        X = check_array(X, accept_sparse='csr')

        Ps = np.zeros((X.shape[0], self.degree, self.n_components))
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                for d in range(self.degree):
                    iHashIndex = self.indexHash_[d, j]
                    iHashBit   = self.bitHash_[d, j]
                    Ps[i, d, iHashIndex] += iHashBit * X[i,j] 

        Ps = fftpack.fft(Ps, axis=2, overwrite_x=True)
        temps = np.prod(Ps, axis=1)
        data_sketch = np.real(fftpack.ifft(temps, overwrite_x=True))

        return data_sketch